<a href="https://colab.research.google.com/github/shahid-jafri/45-Ex/blob/main/Passignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# prompt: Design a Library Management System using Object-Oriented Programming (OOP) concepts and Python's typing features. The system should support basic CRUD operations (Create, Read, Update, Delete) for books, manage different types of users (Librarians and Members), and handle book borrowing transactions with file-based data persistence. Appropriate error handling for file operations is required.
# ---

from typing import List, Dict, Optional
import json
import os

class Book:
    def __init__(self, title: str, author: str, isbn: str, available: bool = True):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.available = available

    def __str__(self):
        return f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, Available: {self.available}"

class User:
    def __init__(self, name: str, user_id: str, user_type: str):
        self.name = name
        self.user_id = user_id
        self.user_type = user_type

    def __str__(self):
        return f"Name: {self.name}, User ID: {self.user_id}, User Type: {self.user_type}"

class Library:
    def __init__(self, data_file: str = "library_data.json"):
        self.data_file = data_file
        self.books: List[Book] = []
        self.users: List[User] = []
        self.borrowed_books: Dict[str, List[str]] = {}  # {user_id: [isbn1, isbn2]}
        self.load_data()

    def load_data(self):
        if os.path.exists(self.data_file):
            try:
                with open(self.data_file, "r") as f:
                    data = json.load(f)
                    for book_data in data.get("books", []):
                        self.books.append(Book(**book_data))
                    for user_data in data.get("users", []):
                        self.users.append(User(**user_data))
                    self.borrowed_books = data.get("borrowed_books", {})
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON data: {e}")
            except FileNotFoundError as e:
                print(f"Error loading data: {e}")

    def save_data(self):
        data = {
            "books": [vars(book) for book in self.books],
            "users": [vars(user) for user in self.users],
            "borrowed_books": self.borrowed_books
        }
        try:
            with open(self.data_file, "w") as f:
                json.dump(data, f, indent=4)
        except Exception as e:
            print(f"Error saving data: {e}")

    def add_book(self, book: Book):
        self.books.append(book)
        self.save_data()

    def remove_book(self, isbn: str):
        for book in self.books:
            if book.isbn == isbn:
                self.books.remove(book)
                self.save_data()
                return True
        return False

    def find_book(self, isbn: str) -> Optional[Book]:
        for book in self.books:
            if book.isbn == isbn:
                return book
        return None

    def add_user(self, user: User):
        self.users.append(user)
        self.save_data()

    def remove_user(self, user_id: str):
        for user in self.users:
            if user.user_id == user_id:
                self.users.remove(user)
                self.save_data()
                return True
        return False

    def borrow_book(self, user_id: str, isbn: str):
        book = self.find_book(isbn)
        if book and book.available:
            book.available = False
            if user_id not in self.borrowed_books:
                self.borrowed_books[user_id] = []
            self.borrowed_books[user_id].append(isbn)
            self.save_data()
            return True
        return False

    def return_book(self, user_id: str, isbn: str):
        book = self.find_book(isbn)
        if book and not book.available and user_id in self.borrowed_books and isbn in self.borrowed_books[user_id]:
            book.available = True
            self.borrowed_books[user_id].remove(isbn)
            self.save_data()
            return True
        return False


# Example Usage:
library = Library()

library.add_book(Book("The Lord of the Rings", "J.R.R. Tolkien", "978-0618640157"))
library.add_book(Book("Pride and Prejudice", "Jane Austen", "978-0141439518"))

library.add_user(User("John Doe", "JD123", "Member"))
library.add_user(User("Jane Smith", "JS456", "Librarian"))

library.borrow_book("JD123", "978-0618640157")

print("Available Books:")
for book in library.books:
    if book.available:
        print(book)

print("\nBorrowed Books:")
for user_id, isbn_list in library.borrowed_books.items():
    user = next((user for user in library.users if user.user_id == user_id), None)
    if user:
        print(f"User: {user.name}")
        for isbn in isbn_list:
            book = library.find_book(isbn)
            if book:
                print(f"  - {book.title}")


Available Books:
Title: Pride and Prejudice, Author: Jane Austen, ISBN: 978-0141439518, Available: True
Title: The Lord of the Rings, Author: J.R.R. Tolkien, ISBN: 978-0618640157, Available: True
Title: Pride and Prejudice, Author: Jane Austen, ISBN: 978-0141439518, Available: True
Title: The Lord of the Rings, Author: J.R.R. Tolkien, ISBN: 978-0618640157, Available: True
Title: Pride and Prejudice, Author: Jane Austen, ISBN: 978-0141439518, Available: True

Borrowed Books:
User: John Doe
  - The Lord of the Rings
